In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import random
import sys
sys.path.append('../tools')

import os

import torch

import pandas as pd
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks
import data_utility, annotation_utility
from models.rns_dataloader import *
from active_learning_utility import get_strategy
from active_learning_data import Data
from active_learning_net import Net
from copy import deepcopy
from models.SwaV import SwaV
from models.SupervisedDownstream import SupervisedDownstream
import warnings
import torch.nn as nn

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
warnings.filterwarnings("ignore", ".*Set a lower value for log_every_n_steps if you want to see logs for the training epoch*")

In [3]:
train_data1 = torch.randn((60,256))
train_data2 = torch.randn((89,256))
train_data3 = torch.randn((35,256))
train_data4 = torch.randn((40,256))
train_data5 = torch.randn((35,256))
train_data6 = torch.randn((20,256))
train_data7 = torch.randn((10,256))
train_data8 = torch.randn((50,256))

train_label1 = torch.randn((60,2))
train_label2 = torch.randn((89,2))
train_label3 = torch.randn((35,2))
train_label4 = torch.randn((40,2))
train_label5 = torch.randn((35,2))
train_label6 = torch.randn((20,2))
train_label7 = torch.randn((10,2))
train_label8 = torch.randn((50,2))

In [4]:
data_list = [train_data1,train_data2,train_data3,train_data4,train_data5,train_data6,train_data7,train_data8]
label_list = [train_label1,train_label2,train_label3,train_label4,train_label5,train_label6,train_label7,train_label8]

In [5]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence
class SimpleDataset(Dataset):
    """A simple dataset example."""

    def __init__(self, data, label):
        """
        Args:
            data (list): List of data.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = data
        self.label = label

    def __len__(self):
        """Return the total number of data points."""
        return len(self.data)

    def __getitem__(self, idx):
        """Retrieve a single data point at the specified index."""
        x = self.data[idx]
        y = self.label[idx]

        return x,y

# def collate_fn(batch):
#     info = list(zip(*batch))
#     data = info[0]
#     label = info[1]
#
#     print(data)
#     print(label)
#     # data.sort(key=lambda x: len(x), reverse=True)
#     data = pack_sequence(data,enforce_sorted=False)
#     # data.sort(key=lambda x: len(x), reverse=True)
#     label = pack_sequence(label,enforce_sorted=False)
#
#     return data,label
def collate_fn(batch):
    info = list(zip(*batch))
    data = list(info[0])
    label = list(info[1])

    print(data[0].size())
    print(data[1].size())
    print(data[2].size())

    # data.sort(key=lambda x: len(x), reverse=True)
    seq_len = [s.size(0) for s in data]
    # print(seq_len)
    # data.sort(key=lambda x: len(x), reverse=True)

    data = pack_sequence(data,enforce_sorted=False)
    label = pack_sequence(label,enforce_sorted=False)
    # label = pad_sequence(label, batch_first=True).float()
    # # data = data.unsqueeze(-1)
    # label = pack_padded_sequence(label,  batch_first=True,enforce_sorted=True)

    return data, label

In [6]:
a = torch.tensor([1,2,3,4])
b = torch.tensor([5,6,7])
c = torch.tensor([7,8])
d = torch.tensor([9])
train_x = [a, b, c, d]
a = torch.tensor([10,20,30,40])
b = torch.tensor([50,60,70])
c = torch.tensor([70,80])
d = torch.tensor([90])
train_y = [a, b, c, d]

In [7]:
data = SimpleDataset(data_list,label_list)
data_loader = DataLoader(data, batch_size=3, shuffle=True, collate_fn=collate_fn)
# 采用默认的 collate_fn 会报错
#data_loader = DataLoader(data, batch_size=2, shuffle=True)
batch_x, batch_y = iter(data_loader).next()

torch.Size([35, 256])
torch.Size([60, 256])
torch.Size([10, 256])


In [8]:
print(batch_x.data.size())

torch.Size([105, 256])


In [9]:
print(batch_y.data.size())

torch.Size([105, 2])


In [11]:
rnn = nn.LSTM(256, 1024, 1, batch_first=True, bidirectional=True)
ln = nn.Linear(2048, 2)
# h0 = torch.rand(1, 2, 4).float()
# c0 = torch.rand(1, 2, 4).float()
out, (h1, c1) = rnn(batch_x)
out_pad, out_len = pad_packed_sequence(out, batch_first=True)

In [14]:
out_pad.size()

torch.Size([3, 60, 2048])

In [16]:
h1.size()

torch.Size([2, 3, 1024])

In [217]:
class TestModel(L.LightningModule):
    def __init__(self, vocab_size):
        super().__init__()
        self.rnn = nn.LSTM(2048, 1024, 1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(1024, 2)

    def forward(self, x):
        print(x.size())
        x, (_, _) = self.rnn(x)
        x, out_len = pad_packed_sequence(out, batch_first=True)
        x = self.fc(x)



        return x

    def training_step(self, batch, batch_idx):
        batch_x, batch_y = batch
        output = self(batch_x)

        loss = torch.nn.functional.nll_loss(output, target.view(-1))
        return loss

    def configure_optimizers(self):
        return torch.optim.SGD(self.model.parameters(), lr=0.1)

PackedSequence(data=tensor([[-0.0389,  0.0633, -0.2791,  ..., -0.0707,  0.1296,  0.0750],
        [-0.0238, -0.3834,  0.0858,  ..., -0.0675, -0.0851,  0.2169],
        [ 0.1407, -0.4122, -0.1507,  ...,  0.0331,  0.1320,  0.0769],
        ...,
        [-0.1645, -0.4452, -0.2556,  ...,  0.0231,  0.0059,  0.0803],
        [-0.0684,  0.2535,  0.3032,  ...,  0.1625, -0.0538,  0.1373],
        [-0.0341,  0.1125,  0.0395,  ...,  0.0863,  0.0731, -0.0858]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), sorted_indices=tensor([0, 2, 1]), unsorted_indices=tensor([0, 2, 1]))

In [230]:
ln(out_pad.data)

torch.Size([3, 89, 2])

In [231]:
torch.zeros(out_len)

tensor([89, 10, 50])

In [237]:
torch.ones(s,2) for s in out_len], batch_first=True)  

torch.Size([3, 89, 2])

In [210]:
out_pad[1]

tensor([[-0.1353,  0.1171,  0.0591,  ..., -0.0651,  0.2652,  0.0494],
        [-0.3548,  0.1277,  0.0018,  ..., -0.1391,  0.1277, -0.4318],
        [-0.0429,  0.0922, -0.0733,  ..., -0.1177,  0.1244, -0.0559],
        ...,
        [-0.1608,  0.0006, -0.1758,  ...,  0.1924,  0.2358,  0.2687],
        [-0.0807, -0.0302, -0.2945,  ..., -0.0398,  0.2050,  0.1637],
        [ 0.1689, -0.1518, -0.1156,  ...,  0.1064,  0.0400,  0.1942]],
       grad_fn=<SelectBackward0>)